In [ ]:
import json
import shutil
import pandas as pd

In [ ]:
lines = []
split='test'
src_img_dir='./test_images'

with open(split+'.jsonl','r') as f:
    lines = f.read().splitlines()

line_dicts = [json.loads(line) for line in lines]
df = pd.DataFrame(line_dicts)

imgs = df.loc[:,['file_name','image_id','width','height']]
#imgs_dict=imgs.set_index('image_id').T.to_dict('list')
#imgs


In [ ]:
annot_lines = df.loc[:,['image_id','annotations']]
annot_lines.reset_index()

annots = {}

for index, row in annot_lines.iterrows():
    image_id=str(row['image_id'])
    annots[image_id]=[]
    foo=imgs.query("image_id==\'" + image_id+"\'")
    img_w=int(foo['width'].iloc[0])
    img_h=int(foo['height'].iloc[0])
    f=open(split+'/labels/'+image_id+'.txt', 'w')
    for row_jsons in row['annotations']:
        #print(row_jsons)
        for row_json in row_jsons:
            #annots[str(row['image_id'])].append(row_json['adjusted_bbox'])
            x,y,w,h = row_json['adjusted_bbox']
            box=[float(x) / img_w + 0.5 * (float(w) / img_w),
                float(y) / img_h + 0.5 * (float(h) / img_h),
                float(w) / img_w, 
                float(h) / img_h]
            annots[image_id].append(box)
            if all(x >= 0 for x in box):
                box_str = '0 ' + ' '.join(map(str, box))
                f.write(box_str + '\n')
    f.close()
    src_img_path = src_img_dir +'/'+image_id+'.jpg'
    dst_img_path = split+'/images/'+image_id+'.jpg'
    shutil.copy(src_img_path, dst_img_path)